Create pull requests database

In [1]:
pip install pyvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
df = pd.read_csv("final_merged_discussions_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4231 entries, 0 to 4230
Data columns (total 13 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   discussion_title                       4224 non-null   object 
 1   discussion_thread_id                   4157 non-null   object 
 2   discussion_thread_created_at           4154 non-null   object 
 3   discussion_thread_author_id            4154 non-null   object 
 4   discussion_thread_author_affiliation   3172 non-null   object 
 5   discussion_thread_comment_count        4154 non-null   float64
 6   discussion_answered_or_unanswered      4154 non-null   object 
 7   discussion_comment_id                  3693 non-null   object 
 8   discussion_parent_comment_id           2204 non-null   object 
 9   discussion_comment_created_at          2364 non-null   object 
 10  discussion_comment_author              3693 non-null   object 
 11  disc

Code 2: Varied sizes of the spheres

In [3]:
pip install plotly

In [4]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import itertools
import numpy as np

In [5]:
# Read the CSV file
df = pd.read_csv("final_merged_discussions_data.csv")

# Drop rows where affiliation info is missing
df = df.dropna(subset=['discussion_thread_author_affiliation', 'discussion_comment_author_affiliation'])

# Create 'comment_or_reply' column
df['comment_or_reply'] = df.apply(lambda row: 'comment' if row['discussion_thread_comment_count'] > 0 and pd.isna(row['discussion_parent_comment_id']) else 'reply', axis=1)

# Additional calculations for inside/outside consumption and contribution
df['inside_consumption'] = 0
df['outside_consumption'] = 0
df['inside_contribution'] = 0
df['outside_contribution'] = 0

In [6]:
### FINDING THE CONTRIBUTER-CONSUMER RELATIONSHIPS ###

# Create pairs for the first scenario: thread creators and comment authors
pairs_scenario_1 = df[df['comment_or_reply'].isin(['comment', 'reply'])][['discussion_thread_author_affiliation', 'discussion_comment_author_affiliation', 'discussion_thread_id']].values

# Create pairs for the second scenario: comment authors (comment) and comment authors (reply) within the same thread
pairs_scenario_2_consumer = df[df['comment_or_reply'] == 'comment'][['discussion_thread_id', 'discussion_comment_author_affiliation']].values
pairs_scenario_2_contributor = df[df['comment_or_reply'] == 'reply'][['discussion_thread_id', 'discussion_comment_author_affiliation']].values

# Ensure both scenarios have the same number of columns
pairs_scenario_2_consumer = np.column_stack((pairs_scenario_2_consumer, np.empty(len(pairs_scenario_2_consumer))))
pairs_scenario_2_contributor = np.column_stack((pairs_scenario_2_contributor, np.empty(len(pairs_scenario_2_contributor))))

# Concatenate pairs from both scenarios
pairs = np.concatenate([pairs_scenario_1, pairs_scenario_2_consumer, pairs_scenario_2_contributor])

# Filter unique thread-consumer pairs using Pandas
# Filter unique thread-consumer pairs
# Convert relevant columns to strings before finding unique pairs
pairs_str = pairs[:, [0, 2, 1]].astype(str)
unique_thread_consumer_pairs = np.unique(pairs_str, axis=0)

# Convert to DataFrame
unique_thread_consumer_pairs_df = pd.DataFrame(unique_thread_consumer_pairs, columns=['discussion_thread_author_affiliation', 'discussion_thread_id', 'discussion_comment_author_affiliation'])
unique_thread_consumer_pairs_df.drop_duplicates(inplace=True)

# Convert back to NumPy array
unique_thread_consumer_pairs = unique_thread_consumer_pairs_df.values

# Filter unique thread-contributor pairs
# Convert relevant columns to strings before finding unique pairs
pairs_str_contributor = pairs[:, [0, 1, 2]].astype(str)
unique_thread_contributor_pairs = np.unique(pairs_str_contributor, axis=0)

# Concatenate both unique pairs
unique_pairs = np.concatenate([unique_thread_consumer_pairs, unique_thread_contributor_pairs])

In [7]:
### INSIDE-OUTSIDE CONTRIBUTION/CONSUMPTION LOGIC ###

# Additional calculations for inside/outside opportunity
df['inside_opportunity'] = 0
df['outside_opportunity'] = 0

# Iterate through pairs and apply the inside-outside contribution/consumption/opportunity logic
for pair in unique_pairs:
    consumer_affiliation, contributor_affiliation, thread_id = pair

    if consumer_affiliation != contributor_affiliation:
        # Different affiliations, increase outside contribution of contributor, outside consumption of consumer
        df.loc[(df['comment_or_reply'].isin(['reply', 'comment'])) & (df['discussion_thread_author_affiliation'] == contributor_affiliation) & (df['discussion_thread_id'] == thread_id), 'outside_contribution'] += 1
        df.loc[(df['comment_or_reply'].isin(['reply', 'comment'])) & (df['discussion_comment_author_affiliation'] == consumer_affiliation) & (df['discussion_thread_id'] == thread_id), 'outside_consumption'] += 1
        # Increase outside opportunity
        df.loc[(df['comment_or_reply'].isin(['reply', 'comment'])) & (df['discussion_comment_author_affiliation'] == consumer_affiliation) & (df['discussion_thread_author_affiliation'] == contributor_affiliation), 'outside_opportunity'] += 1
    else:
        # Same affiliation, increase inside contribution and inside consumption
        df.loc[(df['comment_or_reply'].isin(['reply', 'comment'])) & (df['discussion_thread_author_affiliation'] == contributor_affiliation) & (df['discussion_thread_id'] == thread_id), 'inside_contribution'] += 1
        df.loc[(df['comment_or_reply'].isin(['reply', 'comment'])) & (df['discussion_comment_author_affiliation'] == consumer_affiliation) & (df['discussion_thread_id'] == thread_id), 'inside_consumption'] += 1
        # Increase inside opportunity
        df.loc[(df['comment_or_reply'].isin(['reply', 'comment'])) & (df['discussion_comment_author_affiliation'] == consumer_affiliation) & (df['discussion_thread_author_affiliation'] == contributor_affiliation), 'inside_opportunity'] += 1

# Calculate collaboration index
df['collaboration_index'] = (
    (df['inside_contribution'] / df['inside_opportunity']) /
    (df['outside_contribution'] / df['outside_opportunity'])
)

In [8]:
### INSIDE-OUTSIDE CONTRIBUTION/CONSUMPTION: VISUALIZATION ###

# Group by organization and sum the counts for both contributor and consumer affiliations, including opportunity
org_stats = df.groupby(['discussion_comment_author_affiliation', 'discussion_thread_author_affiliation']).agg({
    'inside_consumption': 'sum',
    'outside_consumption': 'sum',
    'inside_contribution': 'sum',
    'outside_contribution': 'sum',
    'inside_opportunity': 'sum',
    'outside_opportunity': 'sum',
    'collaboration_index': 'mean'  # Include the collaboration index
}).reset_index()

# Combine both contributor and consumer affiliations for x-axis
# Get unique affiliations for both contributors and consumers
unique_affiliations = set(org_stats['discussion_comment_author_affiliation']).union(org_stats['discussion_thread_author_affiliation'])

# Visualizations for organization stats, sorting by consumer-contributor affiliations
fig11 = px.bar(org_stats, x='discussion_comment_author_affiliation', y='inside_consumption',
               labels={'index': 'Organization', 'inside_consumption': 'Inside Consumption'},
               title='Inside Consumption for Each Organization')

fig12 = px.bar(org_stats, x='discussion_comment_author_affiliation', y='outside_consumption',
               labels={'index': 'Organization', 'outside_consumption': 'Outside Consumption'},
               title='Outside Consumption for Each Organization')

fig13 = px.bar(org_stats, x='discussion_comment_author_affiliation', y='inside_contribution',
               labels={'index': 'Organization', 'inside_contribution': 'Inside Contribution'},
               title='Inside Contribution for Each Organization')

fig14 = px.bar(org_stats, x='discussion_comment_author_affiliation', y='outside_contribution',
               labels={'index': 'Organization', 'outside_contribution': 'Outside Contribution'},
               title='Outside Contribution for Each Organization')

fig15 = px.bar(org_stats, x='discussion_comment_author_affiliation', y='collaboration_index',
               labels={'index': 'Organization', 'collaboration_index': 'Collaboration Index'},
               title='Collaboration Index for Each Organization')

# Update x-axis with unique affiliations
fig11.update_xaxes({'categoryorder': 'array', 'categoryarray': list(unique_affiliations)})
fig12.update_xaxes({'categoryorder': 'array', 'categoryarray': list(unique_affiliations)})
fig13.update_xaxes({'categoryorder': 'array', 'categoryarray': list(unique_affiliations)})
fig14.update_xaxes({'categoryorder': 'array', 'categoryarray': list(unique_affiliations)})
fig15.update_xaxes({'categoryorder': 'array', 'categoryarray': list(unique_affiliations)})

# Save plots as interactive HTML files
fig11.write_html("inside_consumption_org.html")
fig12.write_html("outside_consumption_org.html")
fig13.write_html("inside_contribution_org.html")
fig14.write_html("outside_contribution_org.html")
fig15.write_html("collaboration_index_org.html")

In [9]:
# Create arrays to store question and answer counts
question_count = []
answer_count = []

# Loop through unique pairs of user_1 and user_2
for user_1 in df['discussion_thread_author_id'].unique():
    for user_2 in df['discussion_comment_author'].unique():
        # Filter rows for the current pair
        pair_df = df[(df['discussion_thread_author_id'] == user_1) & (df['discussion_comment_author'] == user_2)]

        # Calculate question and answer counts
        question_count.append(pair_df[pair_df['comment_or_reply'] == 'comment'].shape[0])
        answer_count.append(pair_df[pair_df['comment_or_reply'] == 'reply'].shape[0])

# Create a DataFrame to store question and answer counts
weights_df = pd.DataFrame({'question_count': question_count, 'answer_count': answer_count})

# Save the weights DataFrame as CSV
weights_df.to_csv("weights.csv", index=False)

In [10]:
# Identify thread starter, comment starter, and answerer
thread_starters = df[df['discussion_thread_comment_count'] > 0].groupby('discussion_thread_author_id').first().reset_index()
comment_starters = df[df['comment_or_reply'] == 'comment'].groupby('discussion_comment_author').first().reset_index()
answerers = df[df['comment_or_reply'] == 'reply'].groupby('discussion_comment_author').first().reset_index()

# Visualizations for thread starter, comment starter, and answerer
fig_thread_starter = px.bar(thread_starters, x='discussion_thread_author_affiliation', title='Thread Starter by Organization')
fig_comment_starter = px.bar(comment_starters, x='discussion_comment_author_affiliation', title='Comment Starter by Organization')
fig_answerer = px.bar(answerers, x='discussion_comment_author_affiliation', title='Answerer by Organization')

# Save plots as interactive HTML files
fig_thread_starter.write_html("thread_starter_org.html")
fig_comment_starter.write_html("comment_starter_org.html")
fig_answerer.write_html("answerer_org.html")

In [11]:
# Identify contributors and consumers for each user pair
collaboration_stats = df[df['comment_or_reply'] == 'reply'].groupby(['discussion_thread_author_id', 'discussion_comment_author']).size().reset_index(name='collaboration_count')

# Visualizations for collaboration stats
fig_collaboration = px.scatter(collaboration_stats, x='discussion_thread_author_id', y='discussion_comment_author', size='collaboration_count',
                               labels={'discussion_thread_author_id': 'Thread Author', 'discussion_comment_author': 'Comment Author', 'collaboration_count': 'Collaboration Count'},
                               title='Collaboration of One User with Another')

# Save the plot as an interactive HTML file
fig_collaboration.write_html("collaboration_stats.html")

In [12]:
# Contribution and consumption of a specific organization
org_contribution = df.groupby('discussion_comment_author_affiliation')['inside_contribution', 'outside_contribution'].sum().reset_index()
org_consumption = df.groupby('discussion_comment_author_affiliation')['inside_consumption', 'outside_consumption'].sum().reset_index()

# Visualizations for organization contribution and consumption
fig_org_contribution = px.bar(org_contribution, x='discussion_comment_author_affiliation', y=['inside_contribution', 'outside_contribution'],
                               labels={'index': 'Organization', 'value': 'Count'},
                               title='Contribution of Each Organization')

fig_org_consumption = px.bar(org_consumption, x='discussion_comment_author_affiliation', y=['inside_consumption', 'outside_consumption'],
                               labels={'index': 'Organization', 'value': 'Count'},
                               title='Consumption of Each Organization')

# Save plots as interactive HTML files
fig_org_contribution.write_html("org_contribution.html")
fig_org_consumption.write_html("org_consumption.html")

<ipython-input-12-e558fb4c73e6>:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

<ipython-input-12-e558fb4c73e6>:3: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [13]:
# Initialize inside and outside contribution dictionaries for the fantastic collaboration figure
inside_contribution_dict = {}
outside_contribution_dict = {}

# Iterate through rows for the fantastic collaboration figure
for index, row in df.iterrows():
    # Check if it's a comment (consumer-contributor relationship)
    if row['comment_or_reply'] == 'reply':
        consumer_affiliation = row['discussion_comment_author_affiliation']
        contributor_affiliation = row['discussion_thread_author_affiliation']

        # Increment inside or outside contribution based on affiliations
        if consumer_affiliation == contributor_affiliation:
            inside_contribution_dict[consumer_affiliation] = inside_contribution_dict.get(consumer_affiliation, 0) + 1
        else:
            outside_contribution_dict[contributor_affiliation] = outside_contribution_dict.get(contributor_affiliation, 0) + 1

# Create DataFrames for inside and outside contribution for the fantastic collaboration figure
inside_contribution_df = pd.DataFrame(list(inside_contribution_dict.items()), columns=['discussion_comment_author_affiliation', 'inside_contribution'])
outside_contribution_df = pd.DataFrame(list(outside_contribution_dict.items()), columns=['discussion_comment_author_affiliation', 'outside_contribution'])

In [14]:
# Merge inside and outside contribution DataFrames for the fantastic collaboration figure
affiliation_contributions_df = pd.merge(inside_contribution_df, outside_contribution_df, on='discussion_comment_author_affiliation', how='outer')

# Fill NaN values with 0 for the fantastic collaboration figure
affiliation_contributions_df = affiliation_contributions_df.fillna(0)

In [15]:
# Collaboration stats for affiliations for the fantastic collaboration figure
affiliation_collaboration = df[df['comment_or_reply'] == 'reply'].groupby(['discussion_thread_author_affiliation', 'discussion_comment_author_affiliation']).apply(lambda x: pd.Series({
    'inside_contribution': (x['discussion_comment_author_affiliation'] == x['discussion_thread_author_affiliation']).sum(),
    'outside_contribution': (x['discussion_comment_author_affiliation'] != x['discussion_thread_author_affiliation']).sum(),
    'question_count': x['discussion_thread_comment_count'].sum(),
    'answer_count': x['discussion_comment_author'].count()
})).reset_index()

# Fill NaN values with 0 for the fantastic collaboration figure
affiliation_collaboration = affiliation_collaboration.fillna(0)

# Calculate collaboration parameter for the fantastic collaboration figure
affiliation_collaboration['collaboration_parameter'] = (affiliation_collaboration['answer_count'] / affiliation_collaboration['question_count']) / (
    affiliation_collaboration['answer_count'] / affiliation_collaboration['question_count']
)

In [16]:
# Create a complete grid for affiliations for the fantastic collaboration figure
unique_affiliations = df['discussion_comment_author_affiliation'].unique()

In [17]:
affiliation_grid = pd.DataFrame(list(itertools.product(unique_affiliations, repeat=2)), columns=['discussion_thread_author_affiliation', 'discussion_comment_author_affiliation'])

# Merge the grid with collaboration parameters for the fantastic collaboration figure
affiliation_grid = pd.merge(affiliation_grid, affiliation_collaboration, on=['discussion_thread_author_affiliation', 'discussion_comment_author_affiliation'], how='left')

# Replace NaN values with 0 in inside_contribution and outside_contribution columns for the fantastic collaboration figure
affiliation_grid['inside_contribution'] = affiliation_grid['inside_contribution'].fillna(0)
affiliation_grid['outside_contribution'] = affiliation_grid['outside_contribution'].fillna(0)

# Calculate inside and outside collaboration based on the provided logic for the fantastic collaboration figure
affiliation_grid['inside_collaboration'] = (affiliation_grid['answer_count'] / affiliation_grid['question_count']) * affiliation_grid['inside_contribution']
affiliation_grid['outside_collaboration'] = (affiliation_grid['answer_count'] / affiliation_grid['question_count']) * affiliation_grid['outside_contribution']

# Replace NaN values with 0 in inside_collaboration and outside_collaboration columns for the fantastic collaboration figure
affiliation_grid['inside_collaboration'] = affiliation_grid['inside_collaboration'].fillna(0)
affiliation_grid['outside_collaboration'] = affiliation_grid['outside_collaboration'].fillna(0)

In [18]:
# Create a fantastic figure showcasing the contribution of each affiliation with respect to a specific organization
fig_fantastic_collaboration = go.Figure()

# Add scatter plot for inside collaboration for the fantastic collaboration figure
fig_fantastic_collaboration.add_trace(go.Scatter(
    x=affiliation_grid['discussion_thread_author_affiliation'],
    y=affiliation_grid['discussion_comment_author_affiliation'],
    mode='markers',
    marker=dict(
        size=affiliation_grid['inside_collaboration'],
        color=affiliation_grid['inside_collaboration'],
        colorscale='Viridis',
        opacity=0.8
    ),
    text=affiliation_grid['inside_collaboration'],
    hoverinfo='text'
))

# Add scatter plot for outside collaboration for the fantastic collaboration figure
fig_fantastic_collaboration.add_trace(go.Scatter(
    x=affiliation_grid['discussion_thread_author_affiliation'],
    y=affiliation_grid['discussion_comment_author_affiliation'],
    mode='markers',
    marker=dict(
        size=affiliation_grid['outside_collaboration'],
        color=affiliation_grid['outside_collaboration'],
        colorscale='Viridis',
        opacity=0.8
    ),
    text=affiliation_grid['outside_collaboration'],
    hoverinfo='text'
))

# Customize layout for the fantastic collaboration figure
fig_fantastic_collaboration.update_layout(
    title='Fantastic Collaboration Figure',
    xaxis=dict(title='Discussion Thread Author Affiliation'),
    yaxis=dict(title='Discussion Comment Author Affiliation'),
    showlegend=False
)

# Save the fantastic figure as an interactive HTML file for the fantastic collaboration figure
fig_fantastic_collaboration.write_html("fantastic_collaboration.html")

In [19]:
pip install networkx

In [20]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import networkx as nx

In [21]:
# Read the CSV file
df = pd.read_csv("final_merged_discussions_data.csv")

# Drop rows where affiliation info is missing
df = df.dropna(subset=['discussion_thread_author_affiliation', 'discussion_comment_author_affiliation'])

# Create 'comment_or_reply' column
df['comment_or_reply'] = df.apply(lambda row: 'comment' if row['discussion_thread_comment_count'] > 0 and pd.isna(row['discussion_parent_comment_id']) else 'reply', axis=1)

# Additional calculations for inside/outside consumption and contribution
df['inside_consumption'] = 0
df['outside_consumption'] = 0
df['inside_contribution'] = 0
df['outside_contribution'] = 0

In [22]:
### FINDING THE CONTRIBUTER-CONSUMER RELATIONSHIPS ###

# Create pairs for the first scenario: thread creators and comment authors
pairs_scenario_1 = df[df['comment_or_reply'].isin(['comment', 'reply'])][['discussion_thread_author_affiliation', 'discussion_comment_author_affiliation', 'discussion_thread_id']].values

# Create pairs for the second scenario: comment authors (comment) and comment authors (reply) within the same thread
pairs_scenario_2_consumer = df[df['comment_or_reply'] == 'comment'][['discussion_thread_id', 'discussion_comment_author_affiliation']].values
pairs_scenario_2_contributor = df[df['comment_or_reply'] == 'reply'][['discussion_thread_id', 'discussion_comment_author_affiliation']].values

# Ensure both scenarios have the same number of columns
pairs_scenario_2_consumer = np.column_stack((pairs_scenario_2_consumer, np.empty(len(pairs_scenario_2_consumer))))
pairs_scenario_2_contributor = np.column_stack((pairs_scenario_2_contributor, np.empty(len(pairs_scenario_2_contributor))))

# Concatenate pairs from both scenarios
pairs = np.concatenate([pairs_scenario_1, pairs_scenario_2_consumer, pairs_scenario_2_contributor])

# Filter unique thread-consumer pairs using Pandas
# Filter unique thread-consumer pairs
# Convert relevant columns to strings before finding unique pairs
pairs_str = pairs[:, [0, 2, 1]].astype(str)
unique_thread_consumer_pairs = np.unique(pairs_str, axis=0)

# Convert to DataFrame
unique_thread_consumer_pairs_df = pd.DataFrame(unique_thread_consumer_pairs, columns=['discussion_thread_author_affiliation', 'discussion_thread_id', 'discussion_comment_author_affiliation'])
unique_thread_consumer_pairs_df.drop_duplicates(inplace=True)

# Convert back to NumPy array
unique_thread_consumer_pairs = unique_thread_consumer_pairs_df.values

# Filter unique thread-contributor pairs
# Convert relevant columns to strings before finding unique pairs
pairs_str_contributor = pairs[:, [0, 1, 2]].astype(str)
unique_thread_contributor_pairs = np.unique(pairs_str_contributor, axis=0)

# Concatenate both unique pairs
unique_pairs = np.concatenate([unique_thread_consumer_pairs, unique_thread_contributor_pairs])

In [23]:
### INSIDE-OUTSIDE CONTRIBUTION/CONSUMPTION LOGIC ###

# Additional calculations for inside/outside opportunity
df['inside_opportunity'] = 0
df['outside_opportunity'] = 0

# Create a dictionary to store affiliations
affiliation_dict = dict(zip(df['discussion_comment_author_affiliation'], df['discussion_comment_author_affiliation']))
affiliation_dict.update(dict(zip(df['discussion_thread_id'], df['discussion_thread_author_affiliation'])))

In [24]:
# Iterate through pairs and apply the inside-outside contribution/consumption/opportunity logic
for pair in unique_pairs:
    consumer_affiliation, contributor_affiliation, thread_id = pair

    # Check if the affiliations are in the dictionary before accessing them
    if consumer_affiliation in affiliation_dict and contributor_affiliation in affiliation_dict:
        if consumer_affiliation != contributor_affiliation:
            # Different affiliations, increase outside contribution of contributor, outside consumption of consumer
            df.loc[(df['comment_or_reply'].isin(['reply', 'comment'])) & (df['discussion_thread_author_affiliation'] == contributor_affiliation) & (df['discussion_thread_id'] == thread_id), 'outside_contribution'] += 1
            df.loc[(df['comment_or_reply'].isin(['reply', 'comment'])) & (df['discussion_comment_author_affiliation'] == consumer_affiliation) & (df['discussion_thread_id'] == thread_id), 'outside_consumption'] += 1
            # Increase outside opportunity
            if affiliation_dict[consumer_affiliation] == affiliation_dict[contributor_affiliation]:
                df.loc[(df['comment_or_reply'].isin(['reply', 'comment'])) & (df['discussion_comment_author_affiliation'] == consumer_affiliation) & (df['discussion_thread_author_affiliation'] == contributor_affiliation), 'outside_opportunity'] += 1
                #print(df['outside_opportunity'])
        else:
            # Same affiliation, increase inside contribution and inside consumption
            df.loc[(df['comment_or_reply'].isin(['reply', 'comment'])) & (df['discussion_thread_author_affiliation'] == contributor_affiliation) & (df['discussion_thread_id'] == thread_id), 'inside_contribution'] += 1
            df.loc[(df['comment_or_reply'].isin(['reply', 'comment'])) & (df['discussion_comment_author_affiliation'] == consumer_affiliation) & (df['discussion_thread_id'] == thread_id), 'inside_consumption'] += 1
            # Increase inside opportunity
            if affiliation_dict[consumer_affiliation] == affiliation_dict[contributor_affiliation]:
                df.loc[(df['comment_or_reply'].isin(['reply', 'comment'])) & (df['discussion_comment_author_affiliation'] == consumer_affiliation) & (df['discussion_thread_author_affiliation'] == contributor_affiliation), 'inside_opportunity'] += 1
                #print(df['inside_opportunity'])

# Calculate collaboration index
df['collaboration_index'] = (
    (df['inside_contribution'] / df['inside_opportunity']) /
    (df['outside_contribution'] / df['outside_opportunity'])
)

In [25]:
### NETWORK ANALYSIS AND VISUALIZATION ###

# Create a directed graph
G = nx.DiGraph()

# Add nodes for contributors and consumers
contributors = df['discussion_comment_author_affiliation'].unique()
consumers = df['discussion_thread_author_affiliation'].unique()
G.add_nodes_from(contributors, bipartite=0)
G.add_nodes_from(consumers, bipartite=1)

# Add edges with collaboration index as weights
edges = df[['discussion_comment_author_affiliation', 'discussion_thread_author_affiliation', 'collaboration_index']]
edges = edges.rename(columns={'discussion_comment_author_affiliation': 'source', 'discussion_thread_author_affiliation': 'target'})
edges = edges.dropna()  # Drop rows with missing collaboration indexes
edges = edges[edges['collaboration_index'] != 0]  # Drop edges with collaboration index 0
edges = edges[edges['collaboration_index'].notna()]  # Drop rows with NaN collaboration indexes

# Add edges to the graph
G.add_weighted_edges_from(edges.values)

# Get node positions for bipartite layout
pos = {node: (1, i) for i, node in enumerate(contributors)}
pos.update({node: (2, i) for i, node in enumerate(consumers)})

In [26]:
# Create an edge trace
edge_x = []
edge_y = []
edge_weights = []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)
    edge_weights.append(edge[2]['weight'])

# Create edge trace
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='text',
    text=edge_weights,
    mode='lines')

# Create node traces for contributors and consumers
node_contributor_trace = go.Scatter(
    x=[], y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        )
    )
)

node_consumer_trace = go.Scatter(
    x=[], y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        )
    )
)

In [27]:
# Add nodes and edges to the figure
fig_network = go.Figure(data=[edge_trace, node_contributor_trace, node_consumer_trace],
                        layout=go.Layout(
                            showlegend=False,
                            hovermode='closest',
                            margin=dict(b=0, l=0, r=0, t=0),
                            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

# Update node positions
node_positions_contributors = pos
node_positions_consumers = pos
node_contributor_trace['x'] = [node_positions_contributors[node][0] for node in G.nodes() if G.nodes[node]['bipartite'] == 0]
node_contributor_trace['y'] = [node_positions_contributors[node][1] for node in G.nodes() if G.nodes[node]['bipartite'] == 0]
node_contributor_trace['text'] = [f"{node}<br>Collaboration Index: {G.nodes[node].get('collaboration_index', 'N/A')}" for node in G.nodes() if G.nodes[node]['bipartite'] == 0]

node_consumer_trace['x'] = [node_positions_consumers[node][0] for node in G.nodes() if G.nodes[node]['bipartite'] == 1]
node_consumer_trace['y'] = [node_positions_consumers[node][1] for node in G.nodes() if G.nodes[node]['bipartite'] == 1]
node_consumer_trace['text'] = [f"{node}<br>Collaboration Index: {G.nodes[node].get('collaboration_index', 'N/A')}" for node in G.nodes() if G.nodes[node]['bipartite'] == 1]


# Update color scales
node_contributor_trace['marker']['color'] = [G.nodes[node].get('collaboration_index', 0) for node in G.nodes() if G.nodes[node]['bipartite'] == 0]
node_contributor_trace['marker']['colorbar']['tickvals'] = node_contributor_trace['marker']['color']
node_contributor_trace['marker']['colorbar']['ticktext'] = [f"{color_value:.2f}" for color_value in node_contributor_trace['marker']['color']]

node_consumer_trace['marker']['color'] = [G.nodes[node].get('collaboration_index', 0) for node in G.nodes() if G.nodes[node]['bipartite'] == 1]
node_consumer_trace['marker']['colorbar']['tickvals'] = node_consumer_trace['marker']['color']
node_consumer_trace['marker']['colorbar']['ticktext'] = [f"{color_value:.2f}" for color_value in node_consumer_trace['marker']['color']]

# Show the figure
fig_network.show()

In [28]:
# Save collaboration indexes to a CSV file
collaboration_indexes_df = df[['discussion_comment_author_affiliation', 'discussion_thread_author_affiliation', 'collaboration_index']]
collaboration_indexes_df.to_csv("collaboration_indexes.csv", index=False)